In [1]:
#importing all the necessary lib
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer,one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,MaxPooling2D,Reshape,CuDNNLSTM,SpatialDropout1D
from keras.models import Sequential, load_model
from keras import Model,Input
from keras.layers.convolutional import Conv2D,Conv1D
from keras.initializers import he_uniform,he_normal
import keras.backend as k
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from time import time
from keras.regularizers import l1,l2,l1_l2
from scipy.sparse import hstack
from tensorflow.python.keras.callbacks import TensorBoard
import keras
import pickle
import warnings
warnings.filterwarnings("ignore")


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\

In [2]:
#Reading the dataset
project_data = pd.read_csv('processed_train_data.csv')
project_data.head(3)

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,...,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,essay,price,quantity,presence_of_the_numerical_digits
0,0,p253737,c90749f5d961ff158d4b4d1e7dc665fc,mrs,in,2016-12-05 13:43:57,grades_prek_2,literacy_language,esl_literacy,educational support english learners home,...,"\""The limits of your language are the limits o...",NaN,NaN,My students need opportunities to practice beg...,0,0,students english learners working english seco...,154.60,23,0
1,1,p258326,897464ce9ddc600bced1151f324dd63a,mr,fl,2016-10-25 09:22:10,grades_6_8,history_civics_health_sports,civics_government_teamsports,wanted projector hungry learners,...,The projector we need for our school is very c...,NaN,NaN,My students need a projector to help with view...,7,1,students arrive school eager learn polite gene...,299.00,1,0
2,2,p182444,3465aaf82da834c0582ebd0ef8040ca0,ms,az,2016-08-31 12:03:56,grades_6_8,health_sports,health_wellness_teamsports,soccer equipment awesome middle school students,...,The students on the campus come to school know...,NaN,NaN,"My students need shine guards, athletic socks,...",1,0,true champions not always ones win guts mia ha...,516.85,22,0


In [3]:
# adding feature teacher number of previously posted project, presence of numerical digits, price and quantity
project_data.drop(['Unnamed: 0'], axis =1 , inplace = True)
class_label = project_data['project_is_approved']
project_data['remaining_input'] = project_data['teacher_number_of_previously_posted_projects']  +\
                                    project_data['presence_of_the_numerical_digits']  + \
                                    project_data['price'] + project_data['quantity']

In [4]:
project_data['total_txt'] = project_data['project_title'] + ' ' + project_data['essay'] + ' ' + project_data['project_resource_summary']

In [5]:
project_data.replace(to_replace=np.NaN, value= str('nan'),inplace=True)

In [6]:
col = project_data.columns
col

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'essay', 'price', 'quantity', 'presence_of_the_numerical_digits',
       'remaining_input', 'total_txt'],
      dtype='object')

In [7]:
col = ['id','teacher_id','project_submitted_datetime','project_title','project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4','project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved','price', 'quantity',
        'presence_of_the_numerical_digits','essay']

project_data.drop(labels=col,axis =1, inplace=True)

In [8]:
col = project_data.columns

In [9]:
col = ['teacher_prefix', 'school_state', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories','total_txt',
       'remaining_input']
project_data = project_data[col]

In [10]:
def word_ranking(dataframe):
    col_names = dataframe.columns
    features = []
    #performing train test split
    train,test,y_train,y_test = train_test_split(dataframe, class_label , stratify = class_label, train_size = 0.7)

    train,cv,y_train,y_cv = train_test_split(train,y_train,stratify = y_train,train_size = 0.8)
    for col in col_names[5:6]:
        print(col)
        bag_of_words = CountVectorizer(lowercase= False)
        bow_words = bag_of_words.fit_transform(train[col])
        print(bow_words.shape)
        
        #Lets now store the document term matrix in a dictionary.
        freqs = bow_words.sum(axis=0).A1
        index = freqs.argsort()
        words = bag_of_words.get_feature_names()
        
        

        # Assigning Rank to each word based on its freq of occurance. Word with highest freq is assigned rank 1 
        word_rank = dict()
        rank = 1
        for i in index[::-1]:
            k = words[i]
            word_rank[k] = rank
            rank+=1
        features.append(word_rank)

        #Every word in each review is replaced by its rank
        rank = [] # list of all the review with words replaced with rank
        for sent in train[col].values:
            txt_row = []
            for word in sent.split():
                if word in word_rank.keys():
                    txt_row.append(word_rank[word])
                else:
                    pass
            rank.append(txt_row)
        
        train[col] = rank
        
        rank = [] # list of all the review with words replaced with rank
        for sent in test[col].values:
            txt_row = []
            for word in sent.split():
                if word in word_rank.keys():
                    txt_row.append(word_rank[word])
                else:
                    pass
            rank.append(txt_row)
        
        test[col] = rank
        
        rank = [] # list of all the review with words replaced with rank
        for sent in cv[col].values:
            txt_row = []
            for word in sent.split():
                if word in word_rank.keys():
                    txt_row.append(word_rank[word])
                else:
                    pass
            rank.append(txt_row)
        
        cv[col] = rank
    return train,test,cv,y_train,y_test,y_cv,features
            

In [11]:
train,test,cv,y_train,y_test,y_cv,feature_names = word_ranking(project_data)

total_txt
(61178, 56850)


In [12]:
print("Shape of the Train dataset: ", train.shape[0])
print("Shape of the Test dataset: ", test.shape[0])
print("Shape of the CV dataset: ", cv.shape[0])

Shape of the Train dataset:  61178
Shape of the Test dataset:  32775
Shape of the CV dataset:  15295


In [13]:
y_train.shape

(61178,)

In [14]:
#converting class labels to categorical variables
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_cv = to_categorical(y_cv)

# Tokenizing the Text part

In [15]:


max_review_length = 250
X_train = pad_sequences(train['total_txt'], maxlen=max_review_length)  #padding zeros at the begining of each review to make max len as 200
X_test = pad_sequences(test['total_txt'], maxlen=max_review_length)
X_cv = pad_sequences(cv['total_txt'], maxlen=max_review_length)
print(X_train.shape)
print(X_train[256])

(61178, 250)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0  1351    14   427   834   243  2777
  2080    35     1   945     1     2   879  2219    46     2  1929   289
  2219   970  1929  1980     2  6190    29  1050  1450   434  1964  1079
    28    71    28     2  1839  2556  6054  5449     2    28     1  9466
  4313  2199  3114  6054  6054  4047  3108  6054    73  2757    14  2556
     2   959  5655   895     1   269    66  1618     1   328   372    52
    14  3051   383  1050    86   168   243     1   174   539    91  2556
  6054   962  1448     1    78   205  

# Tokenizing the school state

In [16]:
token_sc_stat = CountVectorizer()

# integer encode the documents
school_state_train = token_sc_stat.fit_transform(train['school_state'])
school_state_test = token_sc_stat.transform(test['school_state'])
school_state_cv = token_sc_stat.transform(cv['school_state'])

print(school_state_train.shape)


(61178, 51)


# Tokenizing the project_grade_category

In [17]:
token_grd_cat = CountVectorizer()

# integer encode the documents
project_grade_train = token_grd_cat.fit_transform(train['project_grade_category'])
project_grade_test = token_grd_cat.transform(test['project_grade_category'])
project_grade_cv = token_grd_cat.transform(cv['project_grade_category'])

print(project_grade_train.shape)


(61178, 4)


# Tokenizing the project categories

In [18]:
token_prj_cat = CountVectorizer()

# integer encode the documents
project_cat_train = token_prj_cat.fit_transform(train['project_subject_categories'])
project_cat_test = token_prj_cat.transform(test['project_subject_categories'])
project_cat_cv = token_prj_cat.transform(cv['project_subject_categories'])


print(project_cat_train.shape)


(61178, 51)


# Tokenizing the project subcategories

In [19]:
token_sub_cat = CountVectorizer()

# integer encode the documents
project_subcat_train = token_sub_cat.fit_transform(train['project_subject_subcategories'])
project_subcat_test = token_sub_cat.transform(test['project_subject_subcategories'])
project_subcat_cv = token_sub_cat.transform(cv['project_subject_subcategories'])

print(project_subcat_train.shape)


(61178, 384)


# Tokenizing the teacher prefix

In [20]:
token_pre = CountVectorizer()

# integer encode the documents
teacher_prefix_train = token_pre.fit_transform(train['teacher_prefix'])
teacher_prefix_test = token_pre.transform(test['teacher_prefix'])
teacher_prefix_cv = token_pre.transform(cv['teacher_prefix'])
print(teacher_prefix_train.shape)


(61178, 5)


In [21]:
input2_train = hstack((school_state_train,project_grade_train,project_cat_train,project_subcat_train,teacher_prefix_train,train['remaining_input'][:,None]))

input2_cv = hstack((school_state_cv,project_grade_cv,project_cat_cv,project_subcat_cv,teacher_prefix_cv,cv['remaining_input'][:,None]))
input2_test = hstack((school_state_test,project_grade_test,project_cat_test,project_subcat_test,teacher_prefix_test,test['remaining_input'][:,None]))

In [22]:
input2_train.shape

(61178, 496)

In [23]:
train=input2_train.todense()
test = input2_test.todense()
cv = input2_cv.todense()

In [24]:
train = np.resize(train,new_shape=(61178,495,1))
test =np.resize(test,new_shape=(32775,495,1))
cv = np.resize(cv,new_shape=(15295,495,1))

In [25]:
train.shape

(61178, 495, 1)

# Deep Learning Models

 ### Model 3

In [26]:
dbfile = open('glove_vectors.pickle', 'rb')      
db = pickle.load(dbfile) 

In [27]:
#Getting word vectors with 50 dim
def embedding_mat(word_index,embedding_dim = 300):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = db.get(word)
        if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [28]:
#AUC score 
def auc( y_true, y_pred ) :
    score = tf.py_func( lambda y_true, y_pred : roc_auc_score( y_true, y_pred).astype('float32'),
                        [y_true, y_pred],
                        'float32',
                        stateful=True,
                        name='sklearnAUC' )
    return score

In [30]:
# input 1
input1 = Input(batch_shape=(None,250))
x1 = Embedding(input_dim=56851,output_dim= 300,weights=[embedding_mat(feature_names[0])],trainable = False)(input1)
x1 = SpatialDropout1D(0.3)(x1)
x1 = CuDNNLSTM(256,return_sequences=True)(x1)
x1 = Flatten()(x1)

# input 2
input2 = Input(shape=(495,1))
x2 = Conv1D(filters=64,kernel_size=3,strides=1)(input2)
x2 = Conv1D(filters=64,kernel_size=3,strides=1)(x2)
x2 = Flatten()(x2)

# merging both the inputs
concat = concatenate([x1,x2])
x = Dense(300,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(concat)
x = Dropout(0.4)(x)
x = Dense(256,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.6)(x)
output = Dense(2, activation = 'softmax')(x)
 
# create model with two inputs
model = Model([input1,input2], output)
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.adam(lr=0.0006,decay = 1e-4), metrics=[auc])
print(model.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 250, 300)     17

In [31]:
#model fitting
filepath="weights_3.best_copy.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,tensorboard]
#my_callbacks = [EarlyStopping(monitor='auc', patience=300, verbose=1, mode='max')]
model.fit([X_train,train], y_train, nb_epoch=25,verbose=1,batch_size=256,validation_data=([X_cv,
        cv], y_cv),callbacks = callbacks_list,class_weight = "balanced")

Instructions for updating:
Use tf.cast instead.
Train on 61178 samples, validate on 15295 samples
Epoch 1/25
61178/61178 [==============================] - 73s 1ms/step - loss: 0.7561 - auc: 0.5113 - val_loss: 0.6153 - val_auc: 0.4866

Epoch 00001: val_auc improved from -inf to 0.48661, saving model to weights_3.best_copy.hdf5
Epoch 2/25
61178/61178 [==============================] - 68s 1ms/step - loss: 0.6240 - auc: 0.5085 - val_loss: 0.5780 - val_auc: 0.5093

Epoch 00002: val_auc improved from 0.48661 to 0.50927, saving model to weights_3.best_copy.hdf5
Epoch 3/25
61178/61178 [==============================] - 68s 1ms/step - loss: 0.5712 - auc: 0.5117 - val_loss: 0.5690 - val_auc: 0.5372

Epoch 00003: val_auc improved from 0.50927 to 0.53719, saving model to weights_3.best_copy.hdf5
Epoch 4/25
61178/61178 [==============================] - 68s 1ms/step - loss: 0.5682 - auc: 0.5144 - val_loss: 0.5757 - val_auc: 0.5250

Epoch 00004: val_auc did not improve from 0.53719
Epoch 5/25
6117

# Compiling the model with best parameters : i.e the best weights

In [32]:
# input 1
input1 = Input(batch_shape=(None,250))
x1 = Embedding(input_dim=56851,output_dim= 300,weights=[embedding_mat(feature_names[0])],trainable = False)(input1)
x1 = SpatialDropout1D(0.3)(x1)
x1 = CuDNNLSTM(256,return_sequences=True)(x1)
x1 = Flatten()(x1)

# input 2
input2 = Input(shape=(495,1))
x2 = Conv1D(filters=64,kernel_size=3,strides=1)(input2)
x2 = Conv1D(filters=64,kernel_size=3,strides=1)(x2)
x2 = Flatten()(x2)

# merging both the inputs
concat = concatenate([x1,x2])
x = Dense(300,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(concat)
x = Dropout(0.4)(x)
x = Dense(256,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(128,activation='relu',kernel_initializer=he_normal(),kernel_regularizer=l2(0.0001))(x)
x = Dropout(0.6)(x)
output = Dense(2, activation = 'softmax')(x)
 
# create model with two inputs
model = Model([input1,input2], output)
model.load_weights("weights_3.best_copy.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.adam(lr=0.0006,decay = 1e-4), metrics=[auc])


In [33]:
print("Auc for test data: %0.3f"%roc_auc_score(y_test,model.predict([X_test,test])))
print("Auc for CV data: %0.3f"%roc_auc_score(y_cv,model.predict([X_cv,cv])))
print("Auc for train data: %0.3f"%roc_auc_score(y_train,model.predict([X_train,train])))

Auc for test data: 0.757
Auc for CV data: 0.765
Auc for train data: 0.799


<img src = 'model_3_epoch_auc_loss.jpg'>
<img src = 'model_3_epoch_val_auc_loss.jpg'>